# Book Example: diet
[![diet.ipynb](https://img.shields.io/badge/github-%23121011.svg?logo=github)](https://github.com/ampl/amplcolab/blob/master/book/diet.ipynb) [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ampl/amplcolab/blob/master/book/diet.ipynb) [![Kaggle](https://kaggle.com/static/images/open-in-kaggle.svg)](https://kaggle.com/kernels/welcome?src=https://github.com/ampl/amplcolab/blob/master/book/diet.ipynb) [![Gradient](https://assets.paperspace.io/img/gradient-badge.svg)](https://console.paperspace.com/github/ampl/amplcolab/blob/master/book/diet.ipynb) [![Open In SageMaker Studio Lab](https://studiolab.sagemaker.aws/studiolab.svg)](https://studiolab.sagemaker.aws/import/github/ampl/amplcolab/blob/master/book/diet.ipynb)

Description: book example autogenerated using diet.mod, diet.dat, and diet.run

Tags: ampl-only, ampl-book

Notebook author: N/A

Model author: N/A


In [ ]:
# Install dependencies
!pip install -q amplpy ampltools

In [ ]:
# Google Colab & Kaggle interagration
MODULES=['ampl', 'coin']
from ampltools import cloud_platform_name, ampl_notebook
from amplpy import AMPL, register_magics
if cloud_platform_name() is None:
    ampl = AMPL() # Use local installation of AMPL
else:
    ampl = ampl_notebook(modules=MODULES) # Install AMPL and use it
register_magics(ampl_object=ampl) # Evaluate %%ampl_eval cells with ampl.eval()

### Example: diet

Generated using diet.mod, diet.dat, and diet.run.

Consider the problem of choosing prepared foods to meet certain nutritional requirements.

Sets:
- `NUTR`: set of nutrients to consider
- `FOOD`: set of food to consider

Parameters:
- `cost {FOOD}`: cost of each food
- `f_min {FOOD}`: minimum amount of food to buy
- `f_max {FOOD}`: maximum amount of food to buy
- `n_min {NUTR}`: minimum amount required of each nutrient
- `n_max {NUTR}`: maximum amount allowed of each nutrient
- `amt {NUTR, FOOD}`: amount of each nutrient in each food

Variables:
- `Buy {FOOD}`: amount of food to buy

Objective:
- `Total_Cost`: total cost of the diet

Constraints:
- `Diet {NUTR}`: ensure that the nutritional requirements are satisfied by the diet. 



In [ ]:
%%writefile diet.mod
set NUTR;
set FOOD;

param cost {FOOD} > 0;
param f_min {FOOD} >= 0;
param f_max {j in FOOD} >= f_min[j];

param n_min {NUTR} >= 0;
param n_max {i in NUTR} >= n_min[i];

param amt {NUTR,FOOD} >= 0;

var Buy {j in FOOD} >= f_min[j], <= f_max[j];

minimize Total_Cost:  sum {j in FOOD} cost[j] * Buy[j];

subject to Diet {i in NUTR}:
   n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];



In [ ]:
%%writefile diet.dat
data;

set NUTR := A B1 B2 C ;
set FOOD := BEEF CHK FISH HAM MCH MTL SPG TUR ;

param:   cost  f_min  f_max :=
  BEEF   3.19    0     100
  CHK    2.59    0     100
  FISH   2.29    0     100
  HAM    2.89    0     100
  MCH    1.89    0     100
  MTL    1.99    0     100
  SPG    1.99    0     100
  TUR    2.49    0     100 ;

param:   n_min  n_max :=
   A      700   10000
   C      700   10000
   B1     700   10000
   B2     700   10000 ;

param amt (tr):
           A    C   B1   B2 :=
   BEEF   60   20   10   15
   CHK     8    0   20   20
   FISH    8   10   15   10
   HAM    40   40   35   10
   MCH    15   35   15   15
   MTL    70   30   15   15
   SPG    25   50   25   15
   TUR    60   20   15   10 ;



In [ ]:
%%ampl_eval
model diet.mod;
data diet.dat;
option solver cbc;
solve;
display Buy;

